# Age Detection Model

**Import necessary libraries and prepare the dataset for operations**

In [9]:
import os                  
import cv2                 
import numpy as np
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

import matplotlib.pyplot as plt


image_data=r'C:\Users\lenovo\OneDrive\Desktop\jupyter_Project\age_detect\dataset'      
image_size = 224            # We resize all images to 224x224 (needed for MobileNetV2)
X,y=[],[]

for file in os.listdir(image_data):
    try:
        age=int(file.split("_")[0])
        img_path=os.path.join(image_data,file)
        image=cv2.imread(img_path)
        X.append(image)
        y.append(image)
    except:
        continue

X=np.array(X)
y=np.array(y)

print(f"Total images loaded: {len(X)}")



Total images loaded: 23709


**Split the data for training and testing**

In [10]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)
#print the quantity of training and testing data
print(f"Training data: {len(X_train)}")
print(f"Testing data: {len(X_test)}")

Training data: 18967
Testing data: 4742
